In [14]:
import torch
import numpy as np
import matplotlib.pyplot as pl
# 生成数组集

w_true = torch.tensor([1.2,100,-5.6])
b_true = 66.6
# y = wx+b
def create_example(w,b,example_num): #@save
    X = torch.normal(0,1,(example_num,len(w))) #样本数x特征数 的矩阵
    y = torch.matmul(X,w) + b
    y_e = y + torch.normal(0,0.01,y.shape)
    return X,y.reshape((-1,1)),y_e.reshape((-1,1)) # shape to n x 1 as the old y is a vector nor tensor

(tensor([[-1.3979, -0.3000, -0.0523],
        [ 0.0642, -0.3709,  0.5589],
        [ 0.0171,  2.0539, -0.2210],
        [-0.1464, -1.6133, -0.6053],
        [-0.5650, -1.4901,  0.8100],
        [-1.0519, -0.1044,  1.2435],
        [-0.9771,  0.3107,  1.2169],
        [-1.4691, -1.4918, -0.2967],
        [ 1.6661, -1.4899, -0.0708],
        [-1.3128,  0.4699,  0.2867]]), tensor([[ 35.2194],
        [ 26.4555],
        [273.2469],
        [-91.5178],
        [-87.6236],
        [ 47.9362],
        [ 89.6804],
        [-82.6849],
        [-79.9896],
        [110.4074]]), tensor([ 35.2321,  26.4728, 273.2501, -91.5277, -87.6216,  47.9324,  89.6896,
        -82.6769, -79.9796, 110.4167]))


In [16]:
# 生成 batch
import random
def batch_iter(batch_size,features,labels): #@save
    example_num = len(labels) # the number of data
    indices = list(range(example_num))
    random.shuffle(indices) # randomize indices

    for i in range(0,example_num,batch_size):
        batch_indices = torch.tensor(indices[i:min(i+batch_size,example_num)])
        yield features[batch_indices],labels[batch_indices]

In [38]:
import torch

# 初始化参数
w = torch.normal(0,0.01,size=(3,1),requires_grad=True)
# w = torch.tensor([0.1,0.1,0.1])
b = torch.zeros(1,requires_grad=True)

def model(x:torch.Tensor,w:torch.Tensor,b:torch.Tensor)->torch.Tensor : #@save
    return torch.matmul(x,w) + b
def squared_loss(predict_values:torch.Tensor,labels:torch.Tensor)->torch.tensor:  #@save
    # l = (y_hat - y)^ * 0.5
    return (predict_values-labels.reshape(predict_values.shape))**2 /2 # l = (y - y_ture)^2 / 2


In [39]:
def sgd(params, lr, batch_size): #@save
    """
    date every param in params: b and w
    aram params:
    :param lr:
    :param batch_size:
    :return:
    """
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

lr = 0.03
num_epochs = 10 # update count
net = model
loss = squared_loss
batch_size = 10
features,labels,true_y = create_example(w_true,b,100)
for epoch in range(num_epochs):
    for batch_features, batch_labels in batch_iter(batch_size=batch_size,features=features,labels=labels):
        # iter every batch
        l = loss(net(features,w,b),labels)
        l.sum().backward() #
        sgd([w,b],lr,batch_size)
    with torch.no_grad(): # 在该模块下，所有计算得出的tensor的requires_grad都自动设置为False。
        train_l = loss(net(features,w,b),labels)
        print(f'epch {epoch + 1}: loss:{float(train_l.mean()):f}')

epch 1: loss:2.868914
epch 2: loss:0.012929
epch 3: loss:0.000091
epch 4: loss:0.000001
epch 5: loss:0.000000
epch 6: loss:0.000000
epch 7: loss:0.000000
epch 8: loss:0.000000
epch 9: loss:0.000000
epch 10: loss:0.000000


In [41]:
print(f'w error: {w_true - w.reshape(w_true.shape)}')
print(f'b error: {b_true - b}')

w error: tensor([ 4.7684e-07,  7.6294e-06, -9.5367e-07], grad_fn=<SubBackward0>)
b error: tensor([66.6000], grad_fn=<RsubBackward1>)


In [55]:
from torch.utils import data
# we can also use the utils of torch to generate data batch
def load_array(data_arrays, batch_size, is_train=True): #@save
    """
    :param data_arrays: constructor of features and labels
    :param batch_size:
    :param is_train:
    :return:
    """
    # tensors that have the same size of the first dimension (example number ).
    dataset = data.TensorDataset(*data_arrays) # *unzip data arrays to array params for function
    return data.DataLoader(dataset,batch_size,shuffle=is_train)

tch_size = 10
data_iter = load_array((features,labels),batch_size)
# for i in data_iter:
#     print(i)


In [58]:
from torch import nn
## initial net
linear_model = nn.Linear(2,1) # fully-connected layour
net = nn.Sequential(linear_model) # load lm into net
## initial params
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

## initial loss function
loss = nn.MSELoss()

## create trainer/optimizer
# 小批量随机梯度下降算法 SGD
trainer = torch.optim.SGD(net.parameters(), lr=0.03) # load into params and lr
num_epochs = 10 # update count
features,labels,true_y = create_example(w_true,b,100)
for epoch in range(num_epochs):
    for X,y in data_iter:
        l = loss(net(X),y)
        trainer.zero_grad() # set the grad of all tensor to zero
        l.backward()
        trainer.step() # update the params of "net" by a sgd or other optimizer
    l = loss(net(features),labels)
    print(f'epoch: {epoch+1},loss : {l:f}')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x3 and 2x1)